## 0.0 setup

follow the setup instructions in `1_0_hybridization_setup.ipynb`

## 0.1. imports
### 0.1.1. regular imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

### 0.1.2. local imports

In [ ]:
path_pylcaio: str = '~/github/pylcaio/'
sys.path.append(os.path.join(Path.home(), path_pylcaio)) # required for local import of pylcaio
import pylcaio

## 0.2. file paths
### 0.2.1. directories

In [5]:
%%capture
# home directory
print(path_dir_home := Path.home())
# input directory
print(path_dir_databases := os.path.join(path_dir_home, config['path_dir_databases']))
# output directories
print(path_dir_data := os.path.join(path_dir_home, config['path_dir_data']))
print(path_dir_pylcaio := os.path.join(path_dir_home, path_dir_data, config['path_dir_pylcaio']))
print(path_dir_pymrio := os.path.join(path_dir_home, path_dir_data, config['path_dir_pymrio']))
print(path_dir_e2m := os.path.join(path_dir_home, path_dir_data, config['path_dir_e2m']))

### 0.2.2. files

In [6]:
%%capture
# databases
print(path_exiobase := os.path.join(path_dir_home, path_dir_databases, config['exiobase']))
print(path_dir_ecoinvent := os.path.join(path_dir_home, path_dir_databases, config['ecoinvent']))
# pylcaio output
print(path_pylcaio_database_loader_class_instance := os.path.join(path_dir_pylcaio, config['pylcaio_database_loader_class_instance']))
print(path_pylcaio_class_instance_before_hybrid := os.path.join(path_dir_pylcaio, config['pylcaio_class_instance_before_hybrid']))
print(path_pylcaio_class_instance_after_hybrid := os.path.join(path_dir_pylcaio, config['pylcaio_class_instance_after_hybrid']))
# pymrio output
print(path_pymrio_class_instance := os.path.join(path_dir_pymrio, config['pymrio_class_instance']))

In [ ]:
!mkdir -p $path_dir_data
!mkdir -p $path_dir_e2m
!mkdir -p $path_dir_ecoinvent && cp -ru /srv/data/ecoinvent-3.5-cutoff/* $path_dir_ecoinvent
#!mkdir -p $path_exiobase && cp -ru /srv/data/exiobase_monetary_pxp/* $path_exiobase

## 1.1. read dataframes created by `pymrio` and `ecospold2matrix`

In [ ]:
with open(path_pymrio_class_instance, 'rb') as filestream:
    exiobase: pymrio.core.mriosystem.IOSystem = pd.read_pickle(filestream)

In [ ]:
if load_from_disk == True:
    ecoinvent: dict = read_ecoinvent_pickle(path_file_e2m_pickle)

### 2.1. main `pylcaio` functionality

note that hardcopy might be necessary, because: XXX

In [19]:
database_loader: pylcaio.DatabaseLoader  = pylcaio.DatabaseLoader(
    lca_database_processed = ecoinvent,
    io_database_processed = exiobase,
    lca_database_name_and_version = 'ecoinvent3.5',
    io_database_name_and_version = 'exiobase3')

In [14]:
with open(path_pylcaio_database_loader_class_instance, 'wb') as file_handle:
    pickle.dump(obj = database_loader, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'path_pylcaio_database_loader_class_instance' is not defined

In [21]:
lcaio_object: pylcaio.LCAIO = database_loader.combine_ecoinvent_exiobase(
    complete_extensions = False,
    impact_world = False,
    regionalized = False)

No path for the capital folder was provided. Capitals will not be endogenized


In [22]:
with open(path_pylcaio_class_instance_before_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
lcaio_object.hybridize(
    price_neutral_cut_off_matrix = 'STAM',
    capitals = False,
    priceless_scaling = True)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...


/home/weinold/github/pylcaio_integration_with_brightway/src/pylcaio.py:877: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.H = self.H.append([self.H] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))


Building geography concordance...
Filter H matrix...
Build Cut-off matrix...
Add processes with 'priceless scaling' to Cut-off matrix...


In [24]:
with open(path_pylcaio_class_instance_after_hybrid, 'wb') as file_handle:
    pickle.dump(obj = lcaio_object, file = file_handle, protocol=pickle.HIGHEST_PROTOCOL)